# Demo 16 Latent Variable Modeling with `gensim`

`gensim` (http://radimrehurek.com/gensim) is a library of language processing tools focused on latent variable models of text.

In [17]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import feature_extraction
from gensim import matutils, models

pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

%matplotlib inline
plt.style.use('ggplot')

The data is about sentiments on Amazon reviews.

In [18]:
reviews = []
sentiments = []

with open(os.path.join('..', 'datasets', 'amazon-reviews.txt')) as f:
    for line in f.readlines():
        line = line.strip('\n')
        review, sentiment = line.split('\t')
        sentiment = np.nan if sentiment == '' else int(sentiment)

        reviews.append(review.lower())
        sentiments.append(sentiment)

df = pd.DataFrame({'review': reviews, 'sentiment': sentiments})

In [19]:
df.head()

,review,sentiment
0,i try not to adjust the volume setting to avoi...,NaN
1,so there is no way for me to plug it in here i...,0.0
2,"good case, excellent value.",1.0
3,i thought motorola made reliable products!.,NaN
4,battery for motorola razr.,NaN


In [4]:
df.dropna(inplace = True) # Let's drop the NaN

In [20]:
df.head()

,review,sentiment
0,i try not to adjust the volume setting to avoi...,NaN
1,so there is no way for me to plug it in here i...,0.0
2,"good case, excellent value.",1.0
3,i thought motorola made reliable products!.,NaN
4,battery for motorola razr.,NaN


## LDA with `gensim`

### Let's first translate a set of documents (articles) into a matrix representation with a row per document and a column per feature (word or n-gram)

In [21]:
vectorizer = feature_extraction.text.CountVectorizer(stop_words = 'english')

In [22]:
documents = vectorizer.fit_transform(df.review)

In [23]:
# Let's now build a mapping of numerical ID to word

id2word = dict(enumerate(vectorizer.get_feature_names()))

In [24]:
id2word

{0: u'00',
 1: u'000',
 2: u'00332',
 3: u'00pm',
 4: u'01',
 5: u'02',
 6: u'04',
 7: u'05',
 8: u'06',
 9: u'07',
 10: u'08',
 11: u'09',
 12: u'093744',
 13: u'10',
 14: u'100',
 15: u'1000',
 16: u'1000mah',
 17: u'100k',
 18: u'100m',
 19: u'102',
 20: u'105',
 21: u'109',
 22: u'10meters',
 23: u'10x',
 24: u'11',
 25: u'110',
 26: u'1100',
 27: u'1100mah',
 28: u'112',
 29: u'112kbps',
 30: u'115',
 31: u'118',
 32: u'11mb',
 33: u'11w',
 34: u'12',
 35: u'120',
 36: u'121',
 37: u'128',
 38: u'128k',
 39: u'128mb',
 40: u'12v',
 41: u'13',
 42: u'1300',
 43: u'1301',
 44: u'14',
 45: u'15',
 46: u'150',
 47: u'15g',
 48: u'15th',
 49: u'16',
 50: u'1650sq',
 51: u'16th',
 52: u'17',
 53: u'172x220',
 54: u'175',
 55: u'18',
 56: u'1800',
 57: u'1800beholy',
 58: u'1800mah',
 59: u'181',
 60: u'19',
 61: u'1900',
 62: u'19005',
 63: u'1945',
 64: u'1955',
 65: u'1999',
 66: u'1gb',
 67: u'1st',
 68: u'1x',
 69: u'20',
 70: u'200',
 71: u'2000',
 72: u'2002',
 73: u'2003',
 74: u

### We want to learn which columns are correlated (i.e., likely to come from the same topic).  This is the word distribution.  We can also determine what topics are in each document, the topic distribution.

In [25]:
# First we convert our word-matrix into gensim's format

corpus = matutils.Sparse2Corpus(documents, documents_columns = False)

(Check https://radimrehurek.com/gensim/matutils as needed)

In [26]:
corpus

(Check https://radimrehurek.com/gensim/models/ldamodel as needed)

In [27]:
# Then we fit an LDA model

model = models.ldamodel.LdaModel(corpus = corpus, num_topics = 25, id2word = id2word, passes = 10)

KeyboardInterrupt: 

In this model, we need to explicitly specify the number of topic we want the model to uncover.  This is a critical parameter, but there isn't much guidance on how to choose it.  Try to use domain expertise where possible.

In [ ]:
model

### Goodness of fit

Now we need to assess the goodness of fit for our model.  Like other unsupervised learning techniques, our validation techniques are mostly about interpretation.

Use the following questions to guide you:
- Did we learn reasonable topics?
- Do the words that make up a topic make sense?
- Is this topic helpful towards our goal?

In [ ]:
model.print_topics()

In [13]:
model

NameError: name 'model' is not defined

Some topics will be clearer than others.  The following topics represent clear concepts:
- Cooking and Recipes: 0.009 \* cup + 0.009 \* recipe + 0.007 \* make + 0.007 \* food + 0.006 \* sugar
- Cooking and recipes: 0.013 \* butter + 0.010 \* baking + 0.010 \* dough + 0.009 \* cup + 0.009 \* sugar
- Fashion and Style: 0.013 \* fashion + 0.006 \* like + 0.006 \* dress + 0.005 \* style

## Word2Vec with `gensim`

In [20]:
# Setup the body text
sentences = df.review.map(lambda review: review.split())

In [21]:
sentences

0        [i, try, not, to, adjust, the, volume, setting...
1        [so, there, is, no, way, for, me, to, plug, it...
2                         [good, case,, excellent, value.]
3        [i, thought, motorola, made, reliable, product...
4                          [battery, for, motorola, razr.]
                               ...                        
14999    [the, screen, on, my, phone, said, "not, charg...
15000    [this, is, my, 4th, samsung, cell, phone, with...
15001                                    [great, company.]
15002    [the, "call", and, "hang-up", keys, are, now, ...
15003         [hopefully, the, kyocera, will, be, better!]
Name: review, dtype: object

In [22]:
model = models.Word2Vec(sentences, size = 100, window = 5, min_count = 5, workers = 4)

`Word2Vec` has many arguments:
- `size` represents how many concepts or topics we should use
- `window` represents how many words surrounding a sentence we should use as our original feature
- `min_count` is the number of times that context or word must appear
- `workers` is the number of CPU cores to use to speed up model training

(Check http://radimrehurek.com/gensim/models/word2vec as needed)

In [23]:
model

### Most similar words

The model has a `most_similar` function that helps find the words most similar to the one you queried.  This will return words that are most often used in the same context.

In [24]:
model.most_similar(positive = ['great'])

[('very', 0.9955066442489624),
 ('good', 0.9861261248588562),
 ('works', 0.9809689521789551),
 ('battery', 0.9781995415687561),
 ('as', 0.9767298698425293),
 ('great.', 0.9744351506233215),
 ('awful.', 0.9738140106201172),
 ('sturdy.', 0.9736456274986267),
 ('poor.', 0.9732855558395386),
 ('is', 0.9721172451972961)]

In [25]:
vectorizer.get_feature_names()

[u'00',
 u'000',
 u'00332',
 u'00pm',
 u'01',
 u'02',
 u'04',
 u'05',
 u'06',
 u'07',
 u'08',
 u'09',
 u'093744',
 u'10',
 u'100',
 u'1000',
 u'1000mah',
 u'100k',
 u'100m',
 u'102',
 u'105',
 u'109',
 u'10meters',
 u'10x',
 u'11',
 u'110',
 u'1100',
 u'1100mah',
 u'112',
 u'112kbps',
 u'115',
 u'118',
 u'11mb',
 u'11w',
 u'12',
 u'120',
 u'121',
 u'128',
 u'128k',
 u'128mb',
 u'12v',
 u'13',
 u'1300',
 u'1301',
 u'14',
 u'15',
 u'150',
 u'15g',
 u'15th',
 u'16',
 u'1650sq',
 u'16th',
 u'17',
 u'172x220',
 u'175',
 u'18',
 u'1800',
 u'1800beholy',
 u'1800mah',
 u'181',
 u'19',
 u'1900',
 u'19005',
 u'1945',
 u'1955',
 u'1999',
 u'1gb',
 u'1st',
 u'1x',
 u'20',
 u'200',
 u'2000',
 u'2002',
 u'2003',
 u'2004',
 u'2005',
 u'2006',
 u'2007',
 u'2008',
 u'2008update',
 u'200mhz',
 u'201',
 u'2011',
 u'2012when',
 u'20912',
 u'21',
 u'2125',
 u'215',
 u'2160',
 u'2169011',
 u'22',
 u'225',
 u'2255',
 u'22nd',
 u'23',
 u'24',
 u'240',
 u'2400mah',
 u'2481',
 u'25',
 u'250',
 u'250v',
 u'25k',

In [26]:
sentences

0        [i, try, not, to, adjust, the, volume, setting...
1        [so, there, is, no, way, for, me, to, plug, it...
2                         [good, case,, excellent, value.]
3        [i, thought, motorola, made, reliable, product...
4                          [battery, for, motorola, razr.]
                               ...                        
14999    [the, screen, on, my, phone, said, "not, charg...
15000    [this, is, my, 4th, samsung, cell, phone, with...
15001                                    [great, company.]
15002    [the, "call", and, "hang-up", keys, are, now, ...
15003         [hopefully, the, kyocera, will, be, better!]
Name: review, dtype: object

In [27]:
sentences = list(map(lambda sentence: list(filter(lambda word: word in vectorizer.get_feature_names(), sentence)), sentences))

KeyboardInterrupt: 

In [ ]:
sentences

In [ ]:
model = models.Word2Vec(sentences, size = 100, window = 5, min_count = 5, workers = 4)

In [ ]:
model.most_similar(positive = ['great'])